In [253]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np

# Para visualizar el progreso:
import tqdm

# FUNCIONES SCRAPING:

- FUNCIÓN INFO:

In [249]:
def scrape_info(user_agent, urls):
    
    # Options:
    opts = Options()
    opts.add_argument("--disable-extensions")
    opts.add_argument(f"User-Agent={user_agent}")
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    sleep(5)
    
    info_productos = []
    
    inicio = 1
    for url in urls:
        
        browser.get(url)
        sleep(5)###########################################################################
        
        #Acepto cookies:
        if inicio == 1:            
            browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
            sleep(5)########################################################################
            
        # Desarrollo la página hasta que no de más
        while True:
            try:
                element = browser.find_element(By.CLASS_NAME, "more")
                browser.execute_script("arguments[0].click();", element)
                sleep(5)#####################################################################
            except:
                break    
                
        # Saco la info de cada producto:
        soup = BeautifulSoup(browser.page_source, "html.parser")
        sleep(5)
        
        productos = soup.find("div", class_= "product-grid").find_all("div", class_= "product-card")
        cat = soup.find("h1", class_= "page-title").text.replace("\n", "")
        
        for producto in tqdm.tqdm(productos):
        
            categoria = cat

            try:
                marca = producto.find("div", class_= "product-tile__body").find("div", class_= "product-tile__model").find("p").text
            except:
                marca = "Sin marca"

            try:
                nombre = producto.find("div", class_= "product-tile__body").find("div", class_= "product-tile__name").find("p").text
            except:
                nombre = "Sin nombre"

            try:
                tipo = producto.find("div", class_= "product-tile__body").find("div", class_= "product-tile__description").find("p").text
            except:
                tipo = "Sin categoria"           

            try:    
                especial = producto.find("div", class_= "product-tile__footer").find("div", class_= "swatch-grid-price price").find_all("span", class_= "value")
                
                if len(especial) == 2:
                    precio_sin_desc = especial[0]["content"]
                    precio_con_desc = especial[1]["content"]
                if len(especial) == 1:
                    precio_sin_desc = especial[0]["content"]
                    precio_con_desc = "Sin descuento"
            except:
                precio_sin_desc = "Sin precio"
                precio_con_desc = "Sin precio"
                    
            try:
                descuento = producto.find("div", class_= "product-tile__footer").find("div", class_= "swatch-grid-price price").find("span", class_= "discount").text
            except:
                descuento = "No tiene"
            
            try:
                rating = producto.find("div", class_= "product-tile__body").find("div", class_= "product-tile__rating").find_all("div", class_= "product-rating__star text-icon")
                val = 0
                for estrellas in rating:
                    if not estrellas.find("use")["xlink:href"] == "#iconStar":
                        val +=1
                        
                valoracion = val
            except:
                valoracion = "Sin rating"

            try:
                num_valoraciones = producto.find("div", class_= "product-tile__body").find("div", class_= "product-tile__rating").find("span", class_= "product-rating__reviews-number").text.strip("()")
            except:
                num_valoraciones = "Sin valoraciones"

            info_productos.append([categoria, marca, nombre, tipo, precio_sin_desc, descuento, precio_con_desc, valoracion, num_valoraciones])
        
        
        
        
   
        inicio += 1
    ############################################################################################################    
    browser.close()
    df_info_productos = pd.DataFrame(info_productos, columns= ["categoria", "marca", "nombre", "tipo", "precio_sin_desc", "descuento", "precio_con_desc", "valoracion", "num_valoraciones"])
    return df_info_productos

# Scrapeo la info de todos los productos de cada página:

In [250]:
# PONER EL USER-AGENT Y LOS ELEMENTOS DE BUSQUEDA:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

urls = ["https://www.pacoperfumerias.com/perfumes-hombre", "https://www.pacoperfumerias.com/perfumes-mujer"]

In [251]:
df_info_productos = scrape_info(user_agent, urls)

C:\Users\regue\AppData\Local\Temp\ipykernel_18044\4200724839.py:9: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(executable_path = path_driver, chrome_options=opts)
100%|██████████| 947/947 [00:01<00:00, 834.97it/s]


## CSV:

In [252]:
# df_info_productos.to_csv("datos_perfumes_paco.csv", index="False", sep= ",")

In [213]:
######################################################################################################################################################

In [214]:
######################################################################################################################################################